In [0]:
#@title
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


L' **algoritmo di rilevazione** non è perfetto e si presta ad alcuni errori, ma ha permesso un ampio margine di miglioramento rispetto all'osservazione diretta. 

L' osservazione diretta permetteva di contare con precisione quante persone erano presenti nella piazza in un dato istante, solitamente nel primo pomeriggio.

Il grafico di esempio mostra come alle ore 15.00 sono state contate 25 persone:






In [0]:
#@title

##data

import sqlite3
import pandas as pd
from datetime import datetime
import numpy as np
import altair as alt


observations=pd.DataFrame()
observations['time']=[8.24,8.36,8.48,9.00,9.12,9.24,9.36,9.48,10.00,10.12,10.24,10.36,10.48,11.00,11.12,11.24,11.36,11.48,12.00,12.12,12.24,12.36,12.48,13.00,13.12,13.24,13.36,13.48,14.00,14.12,14.24,14.36,14.48,15.00,15.12,15.24,15.36,15.48,16.00,16.12,16.24,16.36,16.48,17.00,17.12,17.24,17.36,17.48,18.00]
#observations['algoritmo_attuale']=[3,9,5,1,6,7,6,3,1,6,6,3,8,2,5,3,6,4,4,5,7,6,9,6,6,8,6,11,8,8,5,10,12,8,12,12,15,9,11,11,7,11,12,14,10,14,14,13,9]
#observations['algoritmo_con_postprocessing']=[3,9,5,1,8,7,6,3,2,6,6,6,9,3,6,5,6,5,5,6,8,7,9,8,7,11,7,14,12,11,7,11,13,9,14,17,15,10,13,11,8,12,15,15,14,16,15,13,14]
observations['algoritmo_di_rilevazione']=[3,10,9,1,11,8,9,8,2,9,8,10,10,11,6,8,7,8,5,7,10,8,8,9,9,15,10,14,19,19,14,23,21,27,24,28,25,19,27,23,22,34,32,31,30,34,35,36,34]
observations['persone_realmente_presenti']=[3,9,10,1,10,10,11,5,3,8,7,8,10,11,8,7,6,8,5,7,9,8,9,8,9,14,9,16,18,18,15,22,16,25,23,26,22,18,20,24,23,37,30,36,31,36,39,35,39]
observations['osservazione_singola']=25 #observations.loc[observations['time']== 15.00, 'persone_realmente_presenti']
observations['media_reale']=observations['persone_realmente_presenti'].mean()
observations['media_algoritmo']=observations['algoritmo_di_rilevazione'].mean()


observations['time']=observations['time'].astype(str)
observations['time']=pd.to_datetime(observations['time'], format='%H.%M')#.dt.hour

#observations.info()

##viz

import altair as alt

data= observations.melt('time')
#overlay=pd.DataFrame({'x': ['13.48']})
#overlay['x']=pd.to_datetime(overlay['x'], format='%H.%M')

annotazione=pd.DataFrame()
annotazione['time']=['15.00']
annotazione['obs']=[25]
annotazione['time']=pd.to_datetime(annotazione['time'], format='%H.%M')

domain = ['algoritmo_di_rilevazione:Q', 'persone_realmente_presenti:Q', 'osservazione_singola:Q','media_reale:Q', 'media_algoritmo:Q']
range_ = ['orange','brown', 'red', 'blue']

lines = alt.Chart(data).mark_line(opacity=0.9
                                  ).encode(
  x= alt.X('time:T', axis=alt.Axis(title='ora rilevazione')),
  y= alt.Y('value', axis=alt.Axis(title='numero persone')),
  color= alt.Color('variable',
                   scale=alt.Scale(scheme='category10'),
                   sort=['persone_realmente_presenti','osservazione_singola'],
                   legend=alt.Legend(title='aaaaa')
                   ), #(domain=domain,range=range_))
  #tooltip=['variable', 'value', 'time']
  ).interactive(bind_y=True)



annotazione_sing=alt.Chart(annotazione).mark_bar().encode(
    x='time:T',
    y='obs:Q'
)


#rule = alt.Chart(overlay).mark_rule(color='black').encode(x='x:T')
#rule_rilevazione_singola = alt.Chart(overlay_y).mark_rule(color='red').encode(y='rilevazione_singola:Q')
#rule_media_alg = alt.Chart(observations).mark_rule(color='green').encode(y='mean(persone_realmente_presenti):Q')

##viz opzionali
area_errore_alg= alt.Chart(observations).mark_area(color= 'green', opacity=0.7).encode(
    x='time:T',
    y='persone_realmente_presenti:Q',
    y2='algoritmo_di_rilevazione'
)

area_errore_sing= alt.Chart(observations).mark_area(color= '#f7c759',opacity=0.6).encode(
    x='time:T',
    y='osservazione_singola:Q',
    y2='persone_realmente_presenti:Q'
)


(lines
# + annotazione_sing
#+ rule_media_alg
#+ area_errore_sing
#+ area_errore_alg 
).properties(width=900,height=500)




alt.Chart(...)

Per quanto preciso, il dato dell'osservazione diretta fornisce una stima globalmente inesatta della frequentazione della piazza durante la giornata.
La media reale della frequentazione durante la giornata è infatti di 16 persone.

L' algoritmo di rilevazione, sebbene nelle singole rilevazioni può presentare un discreto numero  di errori, permette di ricavere delle stime globalmente più esatte.

I grafici seguenti confrontano l'errore della stima dervivata dall'osservazione diretta rispetto all'errore dell'algoritmo di rilevazione. Il primo fornisce un dato medio, il secondo confronta l'area compresa tra i valori reali e le stime con le due metodologie:



In [0]:
#@title

#dati scarti
scarto_medio_alg=observations['persone_realmente_presenti'] - observations['algoritmo_di_rilevazione']
scarto_medio_alg= scarto_medio_alg.abs().mean()

scarto_medio_sing=observations['persone_realmente_presenti'] - observations['osservazione_singola']
scarto_medio_sing= scarto_medio_sing.abs().mean()

scarti = pd.DataFrame({
    'modalità_osservazione': ['algoritmo_di_rilevazione','osservazione_diretta'],
    'errore_medio': [scarto_medio_alg,  scarto_medio_sing]
})

scarti= scarti.melt('errore_medio')

#viz scarti

domain=['algoritmo_di_rilevazione','osservazione_diretta']
range_=['green','orange']

errore_medio = alt.Chart(scarti).mark_bar().encode(
    y=alt.Y('value:N', axis=alt.Axis(title='metodologia')),
    x='errore_medio',
    color=alt.Color('value:N',scale=alt.Scale(domain=domain,range=range_), legend=None) #alt.Legend(title='Records in Selection'))
).interactive()


errore_medio.properties(width=500,height=200)

alt.Chart(...)

In [0]:
#@title
#(lines + area_errore_sing).properties(width=900,height=500) | (lines + area_errore_alg).properties(width=900,height=500) 



---
## frequentazione piazza


In [0]:
#@title

cnx = sqlite3.connect('/content/gdrive/My Drive/Colab Notebooks/203_ZG/db.sqlite3')

df_gender = pd.read_sql_query("SELECT id, gender FROM app_sentiment_gender_detected", cnx)

df_gender_total=pd.DataFrame()
df_gender_total['genere']=['donna','uomo']
df_gender_total['percentuale']=[
                        df_gender.loc[df_gender['gender'] == 'woman', 'gender'].count(),
                        df_gender.loc[df_gender['gender'] == 'man', 'gender'].count()
                        ]
df_gender_total['percentuale']=df_gender_total['percentuale']/df_gender_total['percentuale'].sum()*100


#viz
grafico_gender= alt.Chart(df_gender_total).mark_bar().encode(
    y='percentuale:Q',
    x=alt.Y('genere:N',sort=alt.EncodingSortField(field='percentuale', order='descending')),
    color=alt.Color('genere', scale=alt.Scale(scheme='set1'))
)

grafico_gender.properties(width=500,height=200)




alt.Chart(...)

In [0]:
df_gender.head(10)

NameError: ignored